In [ ]:
import sys
sys.path.append("../")
import numpy as np
from sanchez_ansatz import SanchezAnsatz
from qiskit_aer import Aer
from experiments import get_state
from experiments.densities import get_probability_freqs
from experiments import ExperimentModule
from qiskit import transpile
from qiskit_algorithms.optimizers import SPSA, COBYLA
from scipy.stats import norm
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
num_qubits = 8
state_type = "normal"
state_params = { "loc": 0.5 , "scale": 0.5, "x_points": np.array([0., 1.])}

#### Original Method

In [ ]:

x_points = np.linspace(state_params["x_points"][0], state_params["x_points"][1], 2**num_qubits)

target_state =  get_state(num_qubits=num_qubits, state_type=state_type, state_params=state_params)
#state  = get_state(num_qubits=num_qubits, state_type=state_type, state_params=state_params)
#probs = norm.pdf(x_points, loc=state_params["loc"], scale=state_params["scale"])
#target_state = probs/np.linalg.norm(probs)

computed_eta = 2/state_params["scale"]**2

sanchez_ansatz = SanchezAnsatz(target_state, eps=0.05) #eta=computed_eta
init_params = sanchez_ansatz.init_params

t_sa = transpile(sanchez_ansatz, basis_gates=["cx", "u"])

em = ExperimentModule(
    t_sa,
    SPSA(maxiter=3000),
    target_state=target_state,
    init_params=init_params,
)
result = em.minimize()

In [ ]:
pt_sa = t_sa.assign_parameters(result.x)
sv_sim = Aer.get_backend("statevector_simulator")
job = sv_sim.run(pt_sa)
job_result = job.result()
res_statevector = job_result.get_statevector().data

fid = np.abs(res_statevector.conj() @ target_state)**2

print("Fidelity: ", fid)
print("Fidloss: ", 1 - fid)
print("SanchezAnsatz k0: ", sanchez_ansatz.k0)
print("Computed eta: ", computed_eta)

In [ ]:
print(res_statevector[:5])
print(target_state[:5])

In [ ]:
plt.plot(x_points, target_state)
plt.plot(x_points, res_statevector)
plt.show()

In [ ]:
t_sa.draw("mpl")

#### Modified Method

In [ ]:

#x_points = np.linspace(state_params["x_points"][0], state_params["x_points"][1], 2**num_qubits)

#target_state =  get_state(num_qubits=num_qubits, state_type=state_type, state_params=state_params)
#state  = get_state(num_qubits=num_qubits, state_type=state_type, state_params=state_params)
#probs = norm.pdf(x_points, loc=state_params["loc"], scale=state_params["scale"])
#target_state = probs/np.linalg.norm(probs)

#computed_eta = 2/state_params["scale"]**2

sanchez_ansatz = SanchezAnsatz(target_state, eps=0.05, build_modified=True, use_entanglement=True, entanglement_type="pairwise") #eta=computed_eta,
init_params = sanchez_ansatz.init_params

t_sa = transpile(sanchez_ansatz, basis_gates=["cx", "u"])

em = ExperimentModule(
    t_sa,
    SPSA(maxiter=3000),
    target_state=target_state,
    init_params=init_params,
)
result = em.minimize()

In [ ]:
pt_sa = t_sa.assign_parameters(result.x)
sv_sim = Aer.get_backend("statevector_simulator")
job = sv_sim.run(pt_sa)
job_result = job.result()
res_statevector = job_result.get_statevector().data

fid = np.abs(res_statevector.conj() @ target_state)**2

print("Fidelity: ", fid)
print("Fidloss: ", 1 - fid)
print("SanchezAnsatz k0: ", sanchez_ansatz.k0)
print("Computed eta: ", computed_eta)

In [ ]:
print(res_statevector[:5])
print(target_state[:5])

In [ ]:
plt.plot(x_points, target_state)
plt.plot(x_points, res_statevector)
plt.show()

In [ ]:
t_sa.draw("mpl")